## Linear Regression

In [2]:
import findspark
findspark.init('/home/venkat/Downloads/spark-3.2.0-bin-hadoop3.2')
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('lrex').getOrCreate()

In [4]:
from pyspark.ml.regression import LinearRegression

In [6]:
training = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

21/11/28 20:20:09 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [7]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

21/11/28 20:23:12 WARN NettyRpcEnv: Ignored failure: java.util.concurrent.TimeoutException: Cannot receive any reply from 10.0.2.15:37139 in 10000 milliseconds


In [8]:
lr = LinearRegression(featuresCol='features', labelCol='label', predictionCol='prediction')

In [9]:
lrModel = lr.fit(training)

21/11/28 20:24:20 WARN Instrumentation: [76079c14] regParam is zero, which might cause numerical instability and overfitting.
21/11/28 20:24:21 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [10]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [11]:
lrModel.intercept

0.14228558260358093

In [12]:
training_summary = lrModel.summary

In [13]:
training_summary.meanSquaredError

103.28843028724194

## Fit with Test - Train Split

In [14]:
all_data = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

21/11/28 20:26:33 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [15]:
train_data, test_data = all_data.randomSplit([0.7,0.3])

In [18]:
train_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                358|
|   mean| 0.7679800031793822|
| stddev|  9.840644376400526|
|    min|-26.805483428483072|
|    max|  27.78383192005107|
+-------+-------------------+



In [19]:
correct_model=lr.fit(train_data)

21/11/28 20:27:58 WARN Instrumentation: [b62d88c7] regParam is zero, which might cause numerical instability and overfitting.


In [20]:
test_results = correct_model.evaluate(test_data)

In [24]:
test_results.meanAbsoluteError

9.344678280506116

In [25]:
unlabeled_data = test_data.select('features')

In [26]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [28]:
prediction = correct_model.transform(unlabeled_data)

In [31]:
prediction.show() #now shows the prediction

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...| -2.7737564369753045|
|(10,[0,1,2,3,4,5,...|  1.3303575380760297|
|(10,[0,1,2,3,4,5,...| 0.38033088664030634|
|(10,[0,1,2,3,4,5,...|  2.4307947668918573|
|(10,[0,1,2,3,4,5,...|  2.3299901180222857|
|(10,[0,1,2,3,4,5,...| 0.03808154985380696|
|(10,[0,1,2,3,4,5,...|  1.2068810022436798|
|(10,[0,1,2,3,4,5,...| -1.4481381006408447|
|(10,[0,1,2,3,4,5,...|  0.4694024637545383|
|(10,[0,1,2,3,4,5,...|   3.150648414482349|
|(10,[0,1,2,3,4,5,...|  0.9297435471237476|
|(10,[0,1,2,3,4,5,...|  1.9405357572621114|
|(10,[0,1,2,3,4,5,...|  1.2142115632289219|
|(10,[0,1,2,3,4,5,...| -0.9406046109671596|
|(10,[0,1,2,3,4,5,...|   3.288558631314404|
|(10,[0,1,2,3,4,5,...|  0.2183595624320384|
|(10,[0,1,2,3,4,5,...| -0.9987539241714881|
|(10,[0,1,2,3,4,5,...|   0.985104772239374|
|(10,[0,1,2,3,4,5,...|0.006577027225581178|
|(10,[0,1,2,3,4,5,...| -0.540452